# 🏥 ShifaMind - Google Colab Setup

**One-click setup for running ShifaMind on Google Colab**

This notebook will:
1. Mount your Google Drive (where your data is stored)
2. Clone the ShifaMind repository
3. Install only the dependencies Colab doesn't have
4. Configure paths to your Google Drive data
5. Run the complete pipeline

---

## Prerequisites

Your Google Drive should have data organized like this:

```
My Drive/
└── ShifaMind/
    └── 01_Raw_Datasets/
        └── Extracted/
            ├── umls-2025AA-metathesaurus-full/
            │   └── 2025AA/META/
            │       ├── MRCONSO.RRF
            │       ├── MRDEF.RRF
            │       └── MRSTY.RRF
            ├── icd10cm-CodesDescriptions-2024/
            │   └── icd10cm-codes-2024.txt
            ├── mimic-iv-3.1/
            │   └── (MIMIC-IV files)
            └── mimic-iv-note-2.2/note/
                └── discharge.csv.gz
```

**If your paths are different**, you'll set them in Step 3.

---

## Step 1: Mount Google Drive

This connects your Google Drive to Colab so we can access your data files.

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Verify mount
if os.path.exists('/content/drive/MyDrive'):
    print("✅ Google Drive mounted successfully!")
else:
    print("❌ Drive mount failed. Please try again.")

## Step 2: Clone ShifaMind Repository

This downloads the ShifaMind code from GitHub.

In [ ]:
import os

# Remove existing clone if present
if os.path.exists('/content/ShifaMind_Capstone'):
    print("🗑️  Removing existing ShifaMind directory...")
    !rm -rf /content/ShifaMind_Capstone

# Clone repository
print("📥 Cloning ShifaMind repository...")
!git clone https://github.com/SyedMohammedSameer/ShifaMind_Capstone.git /content/ShifaMind_Capstone

# Change to repo directory
%cd /content/ShifaMind_Capstone

print("\n✅ Repository cloned successfully!")
print("📂 Current directory:", os.getcwd())

## Step 3: Configure Data Paths

**DEFAULT PATHS (Most users can skip this)**

If your data is at `/content/drive/MyDrive/ShifaMind/01_Raw_Datasets/`, just run this cell as-is.

**CUSTOM PATHS**

If your data is elsewhere, modify the `BASE_PATH` variable below:

In [ ]:
import os
from pathlib import Path

# ============================================================================
# CONFIGURE YOUR DATA PATH HERE
# ============================================================================

# Option 1: Use default path (recommended)
BASE_PATH = "/content/drive/MyDrive/ShifaMind"

# Option 2: Use custom path (uncomment and modify if needed)
# BASE_PATH = "/content/drive/MyDrive/YourCustomFolder/ShifaMind"

# ============================================================================
# SET ENVIRONMENT VARIABLE
# ============================================================================

os.environ['SHIFAMIND_BASE_PATH'] = BASE_PATH

# ============================================================================
# VALIDATE PATHS
# ============================================================================

print("🔍 Validating data paths...\n")
print(f"Base Path: {BASE_PATH}")
print("="*70)

# Check critical paths
paths_to_check = {
    "Base Directory": BASE_PATH,
    "Raw Datasets": f"{BASE_PATH}/01_Raw_Datasets",
    "UMLS MRCONSO": f"{BASE_PATH}/01_Raw_Datasets/Extracted/umls-2025AA-metathesaurus-full/2025AA/META/MRCONSO.RRF",
    "ICD-10 Codes": f"{BASE_PATH}/01_Raw_Datasets/Extracted/icd10cm-CodesDescriptions-2024/icd10cm-codes-2024.txt",
    "MIMIC Notes": f"{BASE_PATH}/01_Raw_Datasets/Extracted/mimic-iv-note-2.2/note/discharge.csv.gz",
}

all_exist = True
for name, path in paths_to_check.items():
    exists = Path(path).exists()
    status = "✅" if exists else "❌"
    print(f"{status} {name}")
    if not exists:
        all_exist = False
        print(f"   Missing: {path}")

print("="*70)
if all_exist:
    print("\n🎉 All paths validated! Ready to proceed.")
else:
    print("\n⚠️  Some paths are missing. Please:")
    print("   1. Verify your BASE_PATH is correct")
    print("   2. Ensure data files are uploaded to Google Drive")
    print("   3. Check folder and file names match exactly")
    print("\n   Then re-run this cell.")

## Step 4: Install Dependencies

Colab already has most packages (PyTorch, NumPy, pandas, etc.). We only install what's missing.

In [ ]:
print("📦 Installing missing dependencies...\n")
print("Note: This may take 2-3 minutes.\n")

# Install only packages not in Colab by default
!pip install -q transformers>=4.35.0
!pip install -q gradio>=4.0.0
!pip install -q scispacy>=0.5.3
!pip install -q jsonlines>=3.1.0
!pip install -q plotly>=5.14.0

# Download spacy medical model
print("\n📥 Downloading medical NLP model...")
!python -m spacy download en_core_sci_sm

print("\n✅ All dependencies installed!")

## Step 5: Verify GPU Access

ShifaMind runs much faster on GPU. Let's check if GPU is available.

In [ ]:
import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✅ GPU Available: {gpu_name}")
    print(f"   Memory: {gpu_memory:.1f} GB")
    print(f"\n🚀 Training will use GPU acceleration!")
else:
    print("⚠️  No GPU detected. Training will use CPU (much slower).")
    print("\n💡 To enable GPU in Colab:")
    print("   1. Go to Runtime > Change runtime type")
    print("   2. Select 'T4 GPU' or 'A100 GPU'")
    print("   3. Click Save")
    print("   4. Restart from Step 1")

---

# 🚀 Pipeline Execution

Now that setup is complete, run the ShifaMind pipeline in order.

---

## Pipeline Step 1: Generate Knowledge Base

**Time**: ~5-10 minutes

This parses UMLS and ICD-10 to create a structured medical knowledge base.

In [ ]:
import os

%cd /content/ShifaMind_Capstone

# Ensure environment variable is set
os.environ['SHIFAMIND_BASE_PATH'] = os.getenv('SHIFAMIND_BASE_PATH', '/content/drive/MyDrive/ShifaMind')

print("🏗️  Generating Clinical Knowledge Base...\n")
!python final_knowledge_base_generator.py

print("\n✅ Knowledge base generated!")
print("📄 Output: 03_Models/clinical_knowledge_base.json")

## Pipeline Step 2: Train Model

**Time**: ~6 hours on GPU (T4), ~1 hour on A100

This trains ShifaMind through 3 stages:
- Stage 1: Diagnosis head (3 epochs)
- Stage 2: Concept head (2 epochs)
- Stage 3: Joint fine-tuning (3 epochs)

**⚠️ Important**: This is a long-running process. Make sure:
1. You have GPU enabled
2. Your Colab session won't timeout (keep browser tab active or use Colab Pro)
3. You have enough Google Drive space for model checkpoints (~2GB)

In [ ]:
import os

%cd /content/ShifaMind_Capstone

# Ensure environment variable is set
os.environ['SHIFAMIND_BASE_PATH'] = os.getenv('SHIFAMIND_BASE_PATH', '/content/drive/MyDrive/ShifaMind')

print("🎓 Training ShifaMind Model...\n")
print("This will take several hours. Progress will be shown below.\n")
print("="*70)

!python final_model_training.py

print("\n✅ Training complete!")
print("📦 Model saved to: 03_Models/checkpoints/shifamind_model.pt")

## Pipeline Step 3: Evaluate Model

**Time**: ~10-15 minutes

This computes comprehensive metrics and generates visualizations.

In [ ]:
import os

%cd /content/ShifaMind_Capstone

# Ensure environment variable is set
os.environ['SHIFAMIND_BASE_PATH'] = os.getenv('SHIFAMIND_BASE_PATH', '/content/drive/MyDrive/ShifaMind')

print("📊 Evaluating Model Performance...\n")
!python final_evaluation.py

print("\n✅ Evaluation complete!")
print("📁 Results saved to: 04_Results/experiments/")

## Pipeline Step 4: Launch Interactive Demo

**Time**: Instant

This launches a Gradio web interface where you can test predictions on clinical notes.

**Note**: The demo will create a public URL you can share (valid for 72 hours).

In [ ]:
import os

%cd /content/ShifaMind_Capstone

# Ensure environment variable is set for subprocess
# (The demo will look for model/KB based on this path)
os.environ['SHIFAMIND_BASE_PATH'] = os.getenv('SHIFAMIND_BASE_PATH', '/content/drive/MyDrive/ShifaMind')

print(f"📂 Using BASE_PATH: {os.environ['SHIFAMIND_BASE_PATH']}")
print("🌐 Launching Interactive Demo...\n")

!python final_demo.py

# The Gradio interface will appear below with a public URL

---

# 🔍 Testing & Inference

After training, you can test the model programmatically.

---

## Quick Inference Test

Test the model on a sample clinical note.

In [ ]:
%cd /content/ShifaMind_Capstone

from final_inference import ShifaMindPredictor
import json

# Initialize predictor
print("🔧 Loading ShifaMind model...\n")
predictor = ShifaMindPredictor()

# Sample clinical note
clinical_note = """
72-year-old male with history of COPD presents with 3 days of fever (38.9°C),
productive cough with yellow sputum, and worsening shortness of breath.
Vital signs: BP 130/80, HR 98, RR 24, O2 sat 89% on room air.
Physical exam notable for crackles in right lower lobe and decreased breath sounds.
Chest X-ray shows right lower lobe infiltrate consistent with pneumonia.
Labs: WBC 14,500, elevated CRP.
"""

# Get prediction
print("🔍 Predicting diagnosis...\n")
result = predictor.predict(clinical_note)

# Display results
print("="*70)
print("PREDICTION RESULTS")
print("="*70)
print(f"\n📋 Diagnosis: {result['diagnosis']['name']}")
print(f"   Code: {result['diagnosis']['code']}")
print(f"   Confidence: {result['diagnosis']['confidence']:.1%}")

print(f"\n🔬 Top Supporting Concepts:")
for i, concept in enumerate(result['concepts'][:5], 1):
    print(f"   {i}. {concept['name']} (score: {concept['score']:.3f})")

print(f"\n📚 Clinical Knowledge Retrieved:")
for i, kb_entry in enumerate(result['knowledge_base'][:3], 1):
    print(f"   {i}. {kb_entry['term']}: {kb_entry['definition'][:80]}...")

print("\n" + "="*70)
print("✅ Inference test complete!")

## Batch Inference

Run predictions on multiple clinical notes at once.

In [ ]:
# Sample batch of notes
clinical_notes = [
    "Patient with fever, cough, and lung infiltrate consistent with pneumonia.",
    "Acute heart failure with pulmonary edema and elevated BNP.",
    "Septic patient with hypotension, fever, and elevated lactate."
]

print("🔄 Running batch predictions...\n")
results = predictor.predict_batch(clinical_notes)

print("="*70)
print("BATCH RESULTS")
print("="*70)
for i, result in enumerate(results, 1):
    print(f"\n{i}. {result['diagnosis']['name']} ({result['diagnosis']['confidence']:.1%})")
    print(f"   Top concept: {result['concepts'][0]['name']}")

print("\n" + "="*70)
print("✅ Batch inference complete!")

---

# 💾 Save Model to Google Drive

Copy the trained model back to Google Drive for safekeeping.

---

In [ ]:
import shutil
from pathlib import Path

# Create backup directory on Drive
backup_dir = Path(os.environ['SHIFAMIND_BASE_PATH']) / '03_Models' / 'checkpoints'
backup_dir.mkdir(parents=True, exist_ok=True)

# Copy model files
model_files = [
    'shifamind_model.pt',
    'clinical_knowledge_base.json'
]

print("💾 Backing up model to Google Drive...\n")
for file in model_files:
    src = Path(f'/content/ShifaMind_Capstone/03_Models/checkpoints/{file}')
    if src.exists():
        dst = backup_dir / file
        shutil.copy2(src, dst)
        print(f"✅ Copied {file}")
    else:
        print(f"⚠️  {file} not found")

print(f"\n📁 Backup location: {backup_dir}")
print("\n✅ Backup complete! Your model is safe even if Colab session ends.")

---

# 🎉 Setup Complete!

You've successfully:
- ✅ Set up ShifaMind on Google Colab
- ✅ Generated the clinical knowledge base
- ✅ Trained the model (if you ran the training step)
- ✅ Evaluated performance
- ✅ Tested inference

## Next Steps

1. **Explore Results**: Check the `04_Results/experiments/` folder for metrics and visualizations
2. **Use the Demo**: Run the demo cell above to get an interactive web interface
3. **Integrate into Your App**: Import `ShifaMindPredictor` and use it in your own code

## Troubleshooting

- **Path Errors**: Re-run Step 3 and verify all paths exist
- **Out of Memory**: Use a smaller batch size or upgrade to Colab Pro for more RAM
- **GPU Not Available**: Go to Runtime > Change runtime type > Select GPU
- **Session Timeout**: Use Colab Pro or keep browser tab active during training

## Support

For issues or questions:
- Check the README.md in the repository
- Review the documentation in the `docs/` folder
- Contact: Mohammed Sameer Syed

---

**Built with precision. Designed for transparency. Created for better healthcare.**

---